# Google AlloyDB for PostgreSQL

> [AlloyDB](https://cloud.google.com/alloydb) is a fully managed PostgreSQL compatible database service for your most demanding enterprise workloads. AlloyDB combines the best of Google with PostgreSQL, for superior performance, scale, and availability. Extend your database application to build AI-powered experiences leveraging AlloyDB Langgraph integrations.

This notebook goes over how to use `AlloyDB for PostgreSQL` to store checkpoints with the `AlloyDBSaver` class.

Learn more about the package on [GitHub](https://github.com/googleapis/langchain-google-alloydb-pg-python/).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-alloydb-pg-python/blob/langgraph-base/docs/langgraph_checkpoint.ipynb)

## Prerequisites

This guide assumes familiarity with the following:

- [LangGraph Persistence](https://langchain-ai.github.io/langgraph/concepts/persistence/)
- [Postgresql](https://www.postgresql.org/about/)
- [AlloyDB](https://cloud.google.com/products/alloydb?hl=en#how-it-works)

When creating LangGraph agents, you can also set them up so that they persist their state. This allows you to do things like interact with an agent multiple times and have it remember previous interactions.

## Before You Begin

To run this notebook, you will need to do the following:

 * [Create a Google Cloud Project](https://developers.google.com/workspace/guides/create-project)
 * [Enable the AlloyDB API](https://console.cloud.google.com/flows/enableapi?apiid=alloydb.googleapis.com)
 * [Create a AlloyDB instance](https://cloud.google.com/alloydb/docs/instance-primary-create)
 * [Create a AlloyDB database](https://cloud.google.com/alloydb/docs/database-create)
 * [Add an IAM database user to the database](https://cloud.google.com/alloydb/docs/manage-iam-authn) (Optional)

### 🦜🔗 Library Installation
The integration lives in its own `langchain-google-alloydb-pg` package, so we need to install it.

In [ ]:
%pip install --upgrade --quiet langchain-google-alloydb-pg[langgraph] langgraph langchain-google-vertexai

**Colab only:** Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top.

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 🔐 Authentication
Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

* If you are using Colab to run this notebook, use the cell below and continue.
* If you are using Vertex AI Workbench, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
from google.colab import auth

auth.authenticate_user()

### ☁ Set Your Google Cloud Project
Set your Google Cloud project so that you can leverage Google Cloud resources within this notebook.

If you don't know your project ID, try the following:

* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113).

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "my-project-id"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


## Basic Usage

### Set AlloyDB database values
Find your database values, in the [AlloyDB cluster page](https://console.cloud.google.com/alloydb?_ga=2.223735448.2062268965.1707700487-2088871159.1707257687).

In [ ]:
# @title Set Your Values Here { display-mode: "form" }
REGION = "us-central1"  # @param {type: "string"}
CLUSTER = "my-alloydb-cluster"  # @param {type: "string"}
INSTANCE = "my-alloydb-instance"  # @param {type: "string"}
DATABASE = "my-database"  # @param {type: "string"}

### AlloyDBEngine Connection Pool

One of the requirements and arguments to establish AlloyDB as a Checkpoint memory store is a `AlloyDBEngine` object. The `AlloyDBEngine`  configures a connection pool to your AlloyDB database, enabling successful connections from your application and following industry best practices.

To create a `AlloyDBEngine` using `AlloyDBEngine.from_instance()` you need to provide only 5 things:

1. `project_id` : Project ID of the Google Cloud Project where the AlloyDB instance is located.
1. `region` : Region where the AlloyDB instance is located.
1. `cluster`: The name of the AlloyDB cluster.
1. `instance` : The name of the AlloyDB instance.
1. `database` : The name of the database to connect to on the AlloyDB instance.

By default, [IAM database authentication](https://cloud.google.com/alloydb/docs/manage-iam-authn) will be used as the method of database authentication. This library uses the IAM principal belonging to the [Application Default Credentials (ADC)](https://cloud.google.com/docs/authentication/application-default-credentials) sourced from the envionment.

Optionally, [built-in database authentication](https://cloud.google.com/alloydb/docs/database-users/about) using a username and password to access the AlloyDB database can also be used. Just provide the optional `user` and `password` arguments to `AlloyDBEngine.from_instance()`:

* `user` : Database user to use for built-in database authentication and login
* `password` : Database password to use for built-in database authentication and login.

To connect to your AlloyDB instance from this notebook, you will need to enable public IP on your instance. Alternatively, you can follow [these instructions](https://cloud.google.com/alloydb/docs/connect-external) to connect to an AlloyDB for PostgreSQL instance with Private IP from outside your VPC.
Learn more about [specifying IP types](https://github.com/GoogleCloudPlatform/alloydb-python-connector?tab=readme-ov-file#specifying-ip-address-type).


In [ ]:
from langchain_google_alloydb_pg import AlloyDBEngine
from google.cloud.alloydb.connector import IPTypes

engine = AlloyDBEngine.from_instance(
    project_id=PROJECT_ID,
    region=REGION,
    cluster=CLUSTER,
    instance=INSTANCE,
    database=DATABASE,
    ip_type=IPTypes.PUBLIC,
)

### AlloyDBEngine for AlloyDB Omni
To create an `AlloyDBEngine` for AlloyDB Omni, you will need a connection url. `AlloyDBEngine.from_engine_args` first creates an async engine and then turns it into an `AlloyDBEngine`. Here is an example connection with the `asyncpg` driver:

In [ ]:
# Replace with your own AlloyDB Omni info
connstring = f"postgresql+asyncpg://{OMNI_USER}:{OMNI_PASSWORD}@{OMNI_HOST}:{OMNI_PORT}/{OMNI_DATABASE}"
engine = AlloyDBEngine.from_engine_args(connstring)

### Initialize a table
The `AlloyDBSaver` class requires a database table with a specific schema in order to store the persist LangGraph agents state.
The `AlloyDBEngine` engine has a helper method `init_checkpoint_table()` that can be used to create a table with the proper schema for you.

In [ ]:
engine.init_checkpoint_table()  # Use table_name to customise the table name

#### Optional Tip: 💡
You can also specify a schema name by passing `schema_name` wherever you pass `table_name`. Eg:

```python
SCHEMA_NAME="my_schema"

engine.init_checkpoint_table(
    table_name=TABLE_NAME,
    schema_name=SCHEMA_NAME     # Default: "public"
)
```

### AlloyDBSaver

To initialize the `AlloyDBSaver` class you need to provide only 4 things:

1. `engine` - An instance of a `AlloyDBEngine` engine.
1. `table_name` : Table name that stores the checkpoints (default: "checkpoints").
1. `schema_name` : The schema name where the table is located (default: "public").
1. `serde`: Serializer for encoding/decoding checkpoints (default: None)

In [ ]:
from langchain_google_alloydb_pg import AlloyDBSaver

checkpointer = AlloyDBSaver.create_sync(
    engine,
    # table_name=TABLE_NAME,
    # schema_name=SCHEMA_NAME,
    # serde=None,
)

#### 📔 Example of Checkpointer methods

In [ ]:
write_config = {"configurable": {"thread_id": "1", "checkpoint_ns": ""}}
read_config = {"configurable": {"thread_id": "1"}}

checkpoint = {
    "v": 1,
    "ts": "2024-07-31T20:14:19.804150+00:00",
    "id": "1ef4f797-8335-6428-8001-8a1503f9b875",
    "channel_values": {"my_key": "meow", "node": "node"},
    "channel_versions": {"__start__": 2, "my_key": 3, "start:node": 3, "node": 3},
    "versions_seen": {
        "__input__": {},
        "__start__": {"__start__": 1},
        "node": {"start:node": 2},
    },
    "pending_sends": [],
}

# store checkpoint
checkpointer.put(write_config, checkpoint, {}, {})
# load checkpoint
checkpointer.get(read_config)

## 🔗 Adding persistence to the pre-built create react agent


In [ ]:
from typing import Literal

from langchain_core.tools import tool
from langchain_google_vertexai import ChatVertexAI
from langgraph.prebuilt import create_react_agent


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]
model = ChatVertexAI(project=PROJECT_ID, model_name="gemini-2.0-flash-exp")

graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
config = {"configurable": {"thread_id": "2"}}
res = graph.invoke({"messages": [("human", "what's the weather in sf")]}, config)
print(res)

In [ ]:
# Example of resulting checkpoint config
checkpoint = checkpointer.get(config)